In [1]:
import pandas as pd
import numpy as np
import re
import statistics

Defino todos los import al principio y, a continuación, importo el .csv:

In [2]:
original = pd.read_csv('./GSAF5.csv', engine='python')
original.head()


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
original.shape

(5992, 24)

Creo un nuevo dataframe seleccionando las columnas con las que voy a trabajar y prescindiendo del resto, pero mantengo la columna "original order" ya que tiene valores únicos y podría usarla como identificador si tuviera que referenciar en un futuro datos adicionales a este dataframe de alguna de las columnas eliminadas:

In [4]:
base = original[['original order', 'Date', 'Type', 'Country', 'Area', 'Location', 'Injury', 'Fatal (Y/N)', 'Investigator or Source']].copy()
base.head()


,original order,Date,Type,Country,Area,Location,Injury,Fatal (Y/N),Investigator or Source
0,5993,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016"
1,5992,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Lacerations to hands,N,"Orlando Sentinel, 9/19/2016"
2,5991,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Lacerations to lower leg,N,"Orlando Sentinel, 9/19/2016"
3,5990,17-Sep-16,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Struck by fin on chest & leg,N,"The Age, 9/18/2016"
4,5989,16-Sep-16,Unprovoked,AUSTRALIA,Victoria,Bells Beach,No injury: Knocked off board by shark,N,"The Age, 9/16/2016"


In [5]:
base.shape


(5992, 9)

Compruebo los valores que aparecen en la columna Type:

In [6]:
pd.unique(base['Type']).tolist()


['Unprovoked', 'Boat', 'Provoked', 'Invalid', 'Sea Disaster', 'Boating']

Unifico 'Boat' y 'Boating':

In [7]:
base.replace({'Type': {'Boat': 'Boating'}},  inplace = True)

In [8]:
pd.unique(base['Type']).tolist()

['Unprovoked', 'Boating', 'Provoked', 'Invalid', 'Sea Disaster']

Examino los valores de los type 'Invalid' y 'Sea Disaster' en la columna 'Injury':

In [9]:
type_invalid = base.query('Type == "Invalid"', inplace = False)
type_invalid.head()


,original order,Date,Type,Country,Area,Location,Injury,Fatal (Y/N),Investigator or Source
50,5943,07-Jun-16,Invalid,USA,South Carolina,"Folly Beach, Charleston County","No injury, board damaged",N,"C. Creswell, GSAF"
73,5920,08-Apr-16,Invalid,CAPE VERDE,Boa Vista Island,NaN,"""Serious""",N,L.O.Guttke
75,5918,07-Apr-16,Invalid,USA,Florida,"Corners Beach, Jupiter, Palm Beach County",Fell off board when spinner shark leapt from t...,N,YouTube
81,5912,13-Mar-16,Invalid,USA,California,"Bolsa Chica State Park, Orange County",Board reportedly bumped by shark. No injury,N,"Orange County Register, 3/13/2016"
90,5903,Reported 10-Feb-2016,Invalid,CAYMAN ISLANDS,Grand Cayman,Stingray City Bar,Minor injury to wrist from Southern stingray,N,R. Branson


In [10]:
type_invalid['Injury'].value_counts()



No injury                                                                                                                                                                                                    21
FATAL                                                                                                                                                                                                        16
Probable drowning & scavenging                                                                                                                                                                                8
No details                                                                                                                                                                                                    5
Shark involvement prior to death unconfirmed                                                                                                                            

In [11]:
type_disaster = base.query('Type == "Sea Disaster"', inplace = False)
type_disaster.head()

,original order,Date,Type,Country,Area,Location,Injury,Fatal (Y/N),Investigator or Source
243,5749,28-Dec-14,Sea Disaster,GREECE,NaN,33 nautical miles off Othonoi Island,"Of 9 bodies recovered, one was bitten by a shark",N,"Greek Reporter, 1/13/2015"
365,5628,04-Jan-14,Sea Disaster,JAPAN,Okinawa Prefecture,Off Miyako Island,5 cm bite to left foot,N,"Focus Taiwan, 1/7/2014"
668,5325,30-Jul-11,Sea Disaster,PHILIPPINES,NaN,NaN,The two Malabagos were bitten by sharks but su...,N,"ABS-CBN News, 8/17/2011"
935,5058,08-Feb-09,Sea Disaster,USA,Puerto Rico,Quebradillas,It is probable that all 5 passengers died on i...,Y,"C. Ekstander, GSAF"
966,5027,25-Nov-08,Sea Disaster,PHILIPPINES,Batanes Provine,Luzon Strait,"Of the 20 crew, 4 were bitten by shark. None o...",N,"C. Johansson, GSAF"


In [12]:
type_disaster['Injury'].value_counts()

FATAL                                                                                                                                                                                                  49
FATAL x 2                                                                                                                                                                                               4
Foot severed                                                                                                                                                                                            2
Survived                                                                                                                                                                                                2
Leg bitten                                                                                                                                                                                      

Descarto los type 'Invalid' porque o no está confirmada la implicación de tiburones, o los ataques han sido post-mortem. También descarto los type 'Sea Disaster' porque considero que es natural que si hay un accidente en alta mar, los tiburones se alimenten de lo que encuentren, por lo que me centraré en los casos en que los tiburones se hayan acercado a las playas o la línea de costa expresamente

In [13]:
base.query('Type == "Unprovoked" or Type == "Provoked" or Type == "Boating"', inplace = True)
base.head()
                 

,original order,Date,Type,Country,Area,Location,Injury,Fatal (Y/N),Investigator or Source
0,5993,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016"
1,5992,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Lacerations to hands,N,"Orlando Sentinel, 9/19/2016"
2,5991,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Lacerations to lower leg,N,"Orlando Sentinel, 9/19/2016"
3,5990,17-Sep-16,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Struck by fin on chest & leg,N,"The Age, 9/18/2016"
4,5989,16-Sep-16,Unprovoked,AUSTRALIA,Victoria,Bells Beach,No injury: Knocked off board by shark,N,"The Age, 9/16/2016"


In [14]:
base.shape


(5253, 9)